## Проект 5. Компьютер говорит нет!
#### Цель - построение модели МО для вторичных клиентов банка, для принятия решения о выдаче кредита на основе данных из БКИ и анкет заёмщиков.

## 1. Импорт библиотек

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from pandas import Series
import pandas as pd
import numpy as np
from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from scipy.stats import ttest_ind

# функция, чтобы разбить данные на трейн и тест
from sklearn.model_selection import train_test_split
# наша модель для классификации
from sklearn.linear_model import LogisticRegression

# метрики
from sklearn.metrics import confusion_matrix, log_loss
from sklearn.metrics import auc, roc_auc_score, roc_curve, accuracy_score
from sklearn.metrics import precision_score,recall_score, f1_score

# гиперпараметры
from sklearn.model_selection import GridSearchCV

import warnings; warnings.simplefilter('ignore')

import datetime
from datetime import datetime, timedelta

# нормализация данных
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# фиксируем random_seed для воспроизводимости
RANDOM_SEED = 42

#### Предобработка

In [4]:
def get_boxplot(column): # построение box-plot
    fig, ax = plt.subplots(figsize = (14,4))
    sns.boxplot(x = 'default', y = column,
                data = df, ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for' + column)
    plt.show()
    
# Cоздадим словарь для замены
education_dict = {'SCH' : 1,
                'GRD' : 2,
                'UGR' : 3,
                  'PGR' : 4,
                  'ACD' : 5
                }

#Функция для анализа квантилей
def get_quantile(df, column):
    #вычисляем межквартильный размах:
    iqr = df[column].quantile(0.75) - df[column].quantile(0.25)

    #25-й перцентиль:
    perc25 = df[column].quantile(0.25)
    #75-й перцентиль:
    perc75 = df[column].quantile(0.75)

    #верхняя граница выбросов:
    l = perc75 + 1.5*iqr
    #нижняя граница выбросов:
    f = perc25 - 1.5*iqr

    print('Верхняя граница выбросов:',
          l, 'Нижняя граница выбросов:', f)
    
    #считаем количество выбросов
    print('Количество выбросов:', column, ':',
         df[df[column]<(perc25-1.5*iqr)][column].count() +
          df[df[column]>(perc75+1.5*iqr)][column].count())
    
    
#Функция для замены выбросов на среднее значение:
def get_filling(df, column):
    #вычисляем межквартильный размах:
    iqr = df[column].quantile(0.75) - df[column].quantile(0.25)

    #25-й перцентиль:
    perc25 = df[column].quantile(0.25)
    #75-й перцентиль:
    perc75 = df[column].quantile(0.75)

    #верхняя граница выбросов:
    l = perc75 + 1.5*iqr
    #нижняя граница выбросов:
    f = perc25 - 1.5*iqr
    #всё что выпадает за границы выбросов, заменяем на среднее
    df[column] = df[column].apply(lambda m: df[column].mean() if m < f else m)
    df[column] = df[column].apply(lambda m: df[column].mean() if m > l else m)

# тест Стьюдента
def get_stat_dif(column): 
    cols = df.loc[:, column].value_counts().index[:10]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(df.loc[df.loc[:, column]==comb[0], 'default'],
                    df.loc[df.loc[:, column]==comb[1],'default']). pvalue\
        <=0.1/len(combinations_all): # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки',column)
            break
            
# логарифмируем признак
def get_log(df, column):
    df[column] = df[column].apply(lambda s: np.log(s+1))

## 2. Импорт данных

In [5]:
DATA_DIR = '/kaggle/input/sf-dst-scoring/'
# тренировочный (train, используется для обучения модели)
df_train = pd.read_csv(DATA_DIR+'/train.csv')
# тестовый (test, используется для оценки точности модели)
df_test = pd.read_csv(DATA_DIR+'test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [6]:
# Для корректной обработки данных объединим train и test в один датасет
df_train['sample'] = 1 # помечаем train
df_test['sample'] = 0 # помечаем test
df_test['default'] = 0 # # в тесте у нас нет значения дефолта, мы его должны предсказать, по этому пока просто заполняем нулями

df = df_test.append(df_train, sort=False).reset_index(drop=True)

## 3. Предварительный анализ данных

In [7]:
# Посмотрим, что находится в датасете
display(df.head(5))
df.info()

In [8]:
df.isnull().sum()

Всего данные о 110148 клиентах. Всего 20 переменных, из них 1 - временной ряд, 6 бинарных, 7 категориальных и 8 числовых. Всего пропусков 272 (0.4%), все пропуски в переменной education. client_id уникальный числовой признак, который не несет полезностей. В бинарных признаках наше целевая переменная default и искуственно добавленный признак тренировочной части датасета Train

#### Описания полей датасета

* client_id - идентификатор клиента
* education - уровень образования
* sex - пол заемщика
* age - возраст заемщика
* car - флаг наличия автомобиля
* car_type - флаг автомобиля иномарки
* decline_app_cnt - количество отказанных прошлых заявок
* good_work - флаг наличия “хорошей” работы
* bki_request_cnt - количество запросов в БКИ
* home_address - категоризатор домашнего адреса
* work_address - категоризатор рабочего адреса
* income - доход заемщика
* foreign_passport - наличие загранпаспорта
* sna - связь заемщика с клиентами банка
* first_time - давность наличия информации о заемщике
* score_bki - скоринговый балл по данным из БКИ
* region_rating - рейтинг региона
* app_date - дата подачи заявки
* default - флаг дефолта по кредиту

In [9]:
#бинарные переменные
bin_cols=['sex','car','car_type','good_work','foreign_passport']

#категориальные переменные
cat_cols=['education','home_address','work_address','sna']

#числовые переменные
num_cols=['age', 'decline_app_cnt', 'income', 'bki_request_cnt',
         'score_bki', 'region_rating', 'first_time', 'month']

## 4.Анализ  переменных

### 1. age

In [10]:
df.age.hist(bins = 50)
df.age.describe()

In [11]:
get_quantile(df, 'age')
get_log(df, 'age')
df.age.hist()

In [12]:
#логарифмируем признак
get_log(df, 'age')
df.age.hist()

Age: Распределение логнормальное , выбросов по квартилям нет

## 2. decline_app_cnt

In [13]:
display(df.decline_app_cnt.describe())
df.decline_app_cnt.hist(bins = 20)

In [14]:
df.groupby(by = 'default')['decline_app_cnt'].mean()


В среднем, у людей, не отдавших кредит, количество отклонённых заявок больше.

In [15]:
get_quantile(df, 'decline_app_cnt')

In [16]:
df.decline_app_cnt.value_counts()

In [17]:
df['decline_app_cnt'] = df['decline_app_cnt'].apply(lambda x: 1 if x>1 else x)
df.decline_app_cnt.hist()

 Упростим признак, заменив все значения больше одного на еденицу. В итоге мы получим бинарный признак 0 - отказанных заявок не было, 1 - отказанные заявки были.

In [18]:
#Посмотрим на корреляцию
df.loc[:,'default'].astype('float64').corr(
    df.loc[:,'decline_app_cnt']).astype('float64')

### 3. bki_request_cnt

In [19]:
display(df['bki_request_cnt'].describe())
df['bki_request_cnt'].hist()

Числовой пизнак, показывающий количество запросов в БКИ. У распределения заметен правый хвост. Проверим на наличие выбросов:

In [20]:
get_quantile(df, 'bki_request_cnt')

In [21]:
#Заменим выбросы средним значением
get_filling( df, 'bki_request_cnt')

In [22]:
df['bki_request_cnt'].hist(bins = 20)

### 4. income

In [23]:
display(df['income'].describe())
df['income'].hist(bins = 50)

In [24]:
get_quantile(df, 'income')

In [25]:
#Стандартизируем данные, используя MinMaxScaler
scaler = MinMaxScaler()
income_minmax = scaler.fit_transform(df.loc[:,['income']])
df['income'] = income_minmax
#Смотрим на получившийся результат
df['income'].hist(bins = 20)

income: Распределение логнормальное , выбросов очень много 

### 5.score_bki

In [26]:
display(df.score_bki.describe())
df.score_bki.hist(bins = 20)

In [27]:
get_quantile(df, 'score_bki')

In [28]:
#Выбросы заменим на среднее значение
get_filling(df, 'score_bki')

In [29]:
#Посмотрим на получившееся распределение
df['score_bki'].hist(bins = 20)

In [30]:
#и кореляцию
df.loc[:,'default'].astype('float64').corr(
    df.loc[:,'score_bki']).astype('float64')

score_bki: Распределение нормальное. Выбросов не много

### 6.sna 

In [31]:
df.sna.hist()
#Категориальный признак, 4 уникальных значения, показывает связь заёмщика с клиентами банка

### 7.Client-id

In [32]:
df.client_id.describe()

73799 уникальных значений.

## 8.app_date

In [33]:
df.app_date.value_counts()

In [34]:
#Переведём дату подачи заявки в формат datetime
df['app_date'] = pd.to_datetime(df['app_date'])
#выделим номер месяца в отдельный столбец
df['month'] = df.app_date.apply(lambda s: s.month)
df.month.hist()

In [35]:
#Посмотрим на корреляцию
df.loc[:,'default'].astype('float64').corr(df.loc[:,'month']).astype('float64')

In [36]:
df.drop(['app_date'], inplace = True, axis = 1)

## 9.education

In [37]:
df.education.hist()

In [38]:
#Заменим пропуски на наиболее часто встречающееся значение 'SCH'
df.education.fillna('SCH', inplace = True)
#Создаём категориальный признак, чем выше значение - 
#тем выше уровент образования
df['education'] = df['education'].replace(to_replace = education_dict)

## 9.sex

In [39]:
df.sex.hist()

На графике видно, что женщин больше, чем мужчин

In [40]:
df.groupby(by = 'default')['sex'].hist()

## 10.region_rating

In [41]:
display(df['region_rating'].value_counts())
df['region_rating'].hist()

In [42]:
df.groupby(by = 'default')['region_rating'].hist()

In [43]:
#Стандартизируем данные, используя MinMaxScaler
scaler = MinMaxScaler()
rating_minmax = scaler.fit_transform(df.loc[:,['region_rating']])
df['region_rating'] = rating_minmax

In [44]:
#посмотрим на получившийся результат
df.region_rating.hist()

## 5.Корреляционный анализ для числовых пременных

In [45]:
sns.set(font_scale=1)
plt.subplots(figsize=(10,8))
sns.heatmap(df[num_cols].corr(), annot=True, fmt='.2f')

стоит обратить внимание на кореляцию признаков region_rating и income (0.31), а также score_bki и decline_app_cnt (0.19)

## Значимость категориальных признаков

преобразуем бинарные переменные при помощи LabelEncoder

In [46]:
label_encoder = LabelEncoder()

for column in bin_cols:
    df[column] = label_encoder.fit_transform(df[column])
    
# убедимся в преобразовании    
df.head()

Для оценки значимости категориальных и бинарных переменных будем использовать функцию mutual_info_classif из библиотеки sklearn. Данная функция опирается на непараметрические методы, основанные на оценке энтропии в группах категориальных переменных.

In [47]:
imp_cat = Series(mutual_info_classif(df[bin_cols + cat_cols],
                                     df['default'], discrete_features =True),
                                     index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [48]:
df.head(5)

## Матрица Корреляций

In [49]:
sns.set(font_scale=1)
plt.subplots(figsize=(16,14))
sns.heatmap(df.corr(), annot=True, fmt='.2f', linewidths = 0.1)

Сильная корреляция наблюдается между:

рабочим и домашним адресами - 0,73. Удалим рабочий адрес автомобилем и его типом - 0,7. объеденим оба этих признака в новый, т.к. один параметр напрямую следует из другого sna и first_time - -0.53. Удалим first_time, у него ниже корреляция с ключевой переменной

In [50]:
df['car+car_type'] = df['car']+df['car_type']
#закодируем новую колонку
df['car+car_type'] = label_encoder.fit_transform(df['car+car_type'])

In [51]:
#удаляем признаки
df.drop(['work_address','car','car_type','first_time'], inplace = True, axis = 1)
#Проверим полученный результат
sns.set(font_scale=1)
plt.subplots(figsize=(16,14))
sns.heatmap(df.corr(), annot=True, fmt='.2f', linewidths = 0.1)

Найдём статистически важные для модели параметры:

In [52]:
for column in df.columns:
    get_stat_dif(column)

In [53]:
#Оставим в модели только статистически важные параметры 
df = df[['client_id', 'education', 'sex', 'car+car_type', 'decline_app_cnt', 'good_work',
        'score_bki', 'bki_request_cnt', 'region_rating', 'home_address',
        'income', 'sna', 'foreign_passport','sample', 'month', 'default']]

In [54]:
#логарифмируем признаки
for column in ['sna', 'month', 'bki_request_cnt', 'education', 'car+car_type']:
    df[column] = df[column].apply(lambda s: np.log(s+1))

In [55]:
#нормализуем признак score_bki
score_minmax = scaler.fit_transform(df.loc[:,['score_bki']])
df['score_bki'] = score_minmax

In [56]:
#Посмотрим на полученнные данные
df.head(10)

## Строим модель

In [57]:
# Теперь выделим часть для обучения модели (research) и финального предсказания (prediction)
research = df.query('sample == 1').drop(['sample'], axis=1)
prediction = df.query('sample == 0').drop(['sample'], axis=1)
X = research.drop(['default', 'client_id'], axis = 1)
Y = research['default'].values

In [58]:
#Разделим данные для обучения следующим образом
X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size=0.20, random_state=42)

In [59]:
#Обучим модель логистической регрессии
lrg = LogisticRegression(penalty = 'l2', C=1.0, 
                         class_weight= 'balanced', random_state = 42)
lrg.fit(X_train, Y_train)

## Оценка качества модели

In [60]:
probs = lrg.predict_proba(X_val)
probs = probs[:,1]
Y_predicted = lrg.predict(X_val)
#Строим ROC-кривую
fpr, tpr, threshold = roc_curve(Y_val, probs)
roc_auc = roc_auc_score(Y_val, probs)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [61]:
сf_mtx = confusion_matrix(Y_val, Y_predicted)
print('Confusion matrix: \n', confusion_matrix(Y_val, Y_predicted))
tn, fp, fn, tp = сf_mtx.ravel()

In [62]:
print('Предсказано невозращение кредита клиентом, по факту вернувшим: {} \n\
 или {}% от всех вернувших \n'.format(fp, round((fp/(fp+tn))*100, 2)))
print('Предсказан возврат кредита клиентом, по факту не вернувшим: {} \n\
или {}% от всех не вернувших\n'.format(fn, 
                        round((1-recall_score(Y_val,Y_predicted))*100, 2)))

In [63]:
# Предсказываем целевую переменную на данных из prediction
X_prediction = prediction.drop(['default', 'client_id'],axis=1).values
prediction_target = lrg.predict_proba(X_prediction)[:, 1]
submission = pd.concat([prediction.client_id,pd.Series(prediction_target,name='default')],axis=1)
submission.to_csv('submission.csv', index=False)
submission.shape

In [64]:
submission.info()

In [65]:
submission.to_csv('submission.csv', index=False)

Cтоит отметить, что из-за несбалансированности классов мы получаем модель, которая считает клиентов "хорошими" . В результате чего кредит одобряется почти всем и мы имеем 1799 не вернувших кредит заёмщиков с предсказанием возврата. Ошибка составила 98.47%. Для балансировки модели использовался гиперпараметр class_weight = 'balanced'. В результате модель начала браковать клиентов, способных вернуть кредит. Но при этом ошибка с невозвращёнными кредитами снизилась до 33,17%